In [33]:
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, text, Column, Integer, String, Float, BigInteger



In [34]:

def create_database(engine, dbname):
    with engine.connect() as connection:
        connection.execution_options(isolation_level="AUTOCOMMIT")
        connection.execute(text(f"CREATE DATABASE {dbname}"))
        print(f"Database '{dbname}' created successfully!")


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, BigInteger, Numeric

def create_tables(engine):
    metadata = MetaData()
    
    departments = Table('departments', metadata,
                        Column('department_id', Integer, primary_key=True),
                        Column('department', String)
                        )
    
    aisles = Table('aisles', metadata,
                   Column('aisle_id', Integer, primary_key=True),
                   Column('aisle', String)
                   )
    
    instacart = Table('instacart', metadata,
                      Column('order_id', BigInteger),
                      Column('user_id', Integer),
                      Column('order_number', Integer),
                      Column('order_dow', Integer),
                      Column('order_hour_of_day', Integer),
                      Column('days_since_prior_order', Float)
                      )
    
    orders = Table('orders', metadata,
                   Column('order_id', BigInteger),
                   Column('product_id', Integer),
                   Column('add_to_cart_order', Float),
                   Column('reordered', Integer)
                   )
    
    products = Table('products', metadata,
                     Column('product_id', Integer),
                     Column('product_name', String),
                     Column('aisle_id', Integer),
                     Column('department_id', Integer),
                     Column('price', Float)
                     )

    metadata.create_all(engine)
    print("Tables created successfully!")



In [ ]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, text

def loadInfo(engine, batch_size=1000):
    try:
        aislesTable = pd.read_csv('C:/DataMining/Proyecto1/CSV/aisles.csv', encoding='utf-8', delimiter=';')
        departmentsTable = pd.read_csv('C:/DataMining/Proyecto1/CSV/departments.csv', encoding='utf-8', delimiter=';')
        instacartTable = pd.read_csv('C:/DataMining/Proyecto1/CSV/instacart_orders.csv', encoding='utf-8', delimiter=';')
        orderproductsTable = pd.read_csv('C:/DataMining/Proyecto1/CSV/order_products.csv', encoding='utf-8', delimiter=';')
        productsTable = pd.read_csv('C:/DataMining/Proyecto1/CSV/products.csv', encoding='utf-8', delimiter=';')
        
        
        csvMap = {
            'aisles': aislesTable,
            'departments': departmentsTable,
            'instacart': instacartTable,
            'orders': orderproductsTable,
            'products': productsTable
        }

        with engine.connect() as connection:
            metadata = MetaData()
            for key, value in csvMap.items():
                table = Table(key, metadata, autoload_with=engine, quote=True)

                for start in range(0, len(value), batch_size):
                    end = start + batch_size
                    batch = value.iloc[start:end]

                    batch_dicts = batch.to_dict(orient='records')

                    with connection.begin() as transaction:
                        try:
                            connection.execute(table.insert(), batch_dicts)
                            transaction.commit()
                        except Exception as e:
                            transaction.rollback()
                            print(f"An error occurred while inserting into table {key}: {e}")
                            raise

                print(f"Inserted data into table: {key}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [37]:
import os

In [ ]:

user= os.getenv('POSTGRES_USER')
password= os.getenv('POSTGRES_PASSWORD')
dbname = 'postgres'
host = 'localhost'
port = '5432'

dbCreated = 'rawproyecto1'


In [ ]:

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/postgres')


In [ ]:
create_database(engine, dbCreated)

Database 'rawproyecto1' created successfully!


In [ ]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbCreated}')

In [42]:
create_tables(engine)

Tables created successfully!


In [43]:

loadInfo(engine)

Inserted data into table: aisles
Inserted data into table: departments
Inserted data into table: instacart
Inserted data into table: orders
Inserted data into table: products
